<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/stock_market_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [4]:
!pip install pyautogen --upgrade
!pip install python-dotenv
!pip install "autogen-ext[openai]"
!pip install "pyautogen[retrievechat]"

# Import Libraries

In [10]:
import autogen
from autogen import ConversableAgent
import os
import json
from dotenv import load_dotenv
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"

Mounted at /content/drive


# Setup

In [11]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [17]:
llm_config = {
    "config_list": [
        {
            "model": 'gpt-4o',
            "api_key": os.environ["OPENAI_API_KEY"],
            "temperature": 0.7,
            "cache_seed": 42
        }
    ]
}

# Conversation

In [28]:
company_name = input("Enter the company name: ").strip()
task = f"Write a blogpost about the stock price performance of {company_name} in the past month. Today's date is 2025-03-12."

user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine what information is needed to complete the task. "
               	"Please note that the information will all be retrieved using Python code. "
               	"After each step is done by others, check the progress and instruct the remaining steps. "
               	"If a step fails, try to work around.",
    description="Planner. Given a task, determine what information is needed to complete the task. "
            	"After each step is done by others, check the progress and instruct the remaining steps.",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan provided by the planner.",
)

executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. Please write blogs in markdown format (with relevant titles) "
               	"and put the content in pseudo ```md``` code block. "
               	"You take feedback from the admin and refine your blog. Please include visualizations.",
    description="Writer. Write blogs based on the code execution results and take feedback from the admin to refine the blog.",
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Enter the company name: Nvidia


Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2025-03-12.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, we need to follow these steps:

1. **Retrieve Historical Stock Data**: 
   - We need to gather Nvidia's stock price data for the past month, from February 12, 2025, to March 11, 2025. This includes the daily opening, closing, high, and low prices, along with the trading volume.

2. **Data Analysis**:
   - Analyze the stock price data to identify trends, such as overall performance (e.g., percentage increase or decrease), significant fluctuations, and any notable trading days.

3. **Contextual Information**:
   - Investigate any relevant news or events during this period that could have influenced Nvidia's stock